In [3]:
# Importing libraries

from kaggle.api.kaggle_api_extended import KaggleApi
import numpy as np
import seaborn as sns 
import matplotlib as mpl
import pandas as pd

In [1]:
# Downloading the Dataset

api = KaggleApi()
api.authenticate()

api.dataset_download_files(
    "ngshiheng/michelin-guide-restaurants-2021",
    path=".",   
    unzip=True 
)

Dataset URL: https://www.kaggle.com/datasets/ngshiheng/michelin-guide-restaurants-2021


In [5]:
data = pd.read_csv('michelin_my_maps.csv')
data.head()

,Name,Address,Location,Price,Cuisine,Longitude,Latitude,PhoneNumber,Url,WebsiteUrl,Award,GreenStar,FacilitiesAndServices,Description
0,ES:SENZ,"Mietenkamer Straße 65, Grassau, 83224, Germany","Grassau, Germany",€€€€,"Creative, Modern Cuisine",12.465618,47.785630,4.986414e+11,https://guide.michelin.com/en/bayern/grassau/r...,https://www.das-achental.com/,3 Stars,0,"Air conditioning,Car park,Interesting wine list","Here in the restaurant of Das Achental hotel, ..."
1,Tohru in der Schreiberei,"Burgstraße 5, Munich, 80331, Germany","Munich, Germany",€€€€,"Modern Cuisine, Japanese Contemporary",11.577475,48.137597,4.989215e+11,https://guide.michelin.com/en/bayern/mnchen/re...,https://schreiberei-muc.de/,3 Stars,0,"Interesting wine list,Notable sake list",It is absolutely worth climbing the 23 steps o...
2,Schwarzwaldstube,"Tonbachstraße 237, Baiersbronn, 72270, Germany","Baiersbronn, Germany",€€€€,"Classic French, Creative",8.358280,48.536911,4.974425e+11,https://guide.michelin.com/en/baden-wurttember...,https://www.traube-tonbach.de/restaurants-bar/...,3 Stars,0,"Air conditioning,Car park,Great view,Interesti...","Schwarzwaldstube, the flagship restaurant of t..."
3,Victor's Fine Dining by christian bau,"Schlossstraße 27, Perl, 66706, Germany","Perl, Germany",€€€€,"Creative, Contemporary",6.387211,49.535173,4.968668e+10,https://guide.michelin.com/en/saarland/perl/re...,https://www.victors-fine-dining.de/,3 Stars,0,"Air conditioning,Car park,Interesting wine lis...",Few chefs could bring together the flavours of...
4,Aqua,"Parkstraße 1, Wolfsburg, 38440, Germany","Wolfsburg, Germany",€€€€,"Creative, Modern Cuisine",10.789979,52.432849,4.953616e+11,https://guide.michelin.com/en/niedersachsen/wo...,https://www.restaurant-aqua.com,3 Stars,0,"Air conditioning,Car park,Interesting wine lis...",The Ritz-Carlton is located in the heart of Vo...
